In [1]:
config = {
    'host': 'localhost',
    'port' : 5432,
    'database': 'mimic_iv',
    'user': 'mimic_read_only_user',
    'password': 'mimiciv',
    'schema': 'hosp'
}

from sqlalchemy import create_engine, text
import pandas as pd



query = """
SELECT 
    p.*,
    CASE 
        WHEN matches[1] IS NOT NULL THEN CAST(matches[1] AS FLOAT)
        ELSE NULL
    END AS quantity,
    CASE 
        WHEN matches[1] IS NOT NULL THEN matches[3]
        ELSE NULL
    END AS unit,
    CASE 
        WHEN matches[1] IS NOT NULL THEN TRIM(BOTH ' ' FROM matches[5])
        ELSE p.prod_strength
    END AS form
FROM 
    hosp.prescriptions p,
    LATERAL REGEXP_MATCHES(p.prod_strength, '^([0-9]+(\.[0-9]+)?)?[ ]?([a-zA-Z]+(/[a-zA-Z]+)?)?([a-zA-Z ]+)$') AS matches
LIMIT 200000
    """





engine = create_engine(f'postgresql://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}')
with engine.connect() as connection:
    df = pd.read_sql_query(text(query), connection)
df



<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
C:\Users\jvoyer\AppData\Local\Temp\5\ipykernel_12172\4022692885.py:15: SyntaxWarning: invalid escape sequence '\.'
  query = """


,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,quantity,unit,form
0,10959054,22224110,62822945,10959054-388,388.0,P38G1F,2185-09-11 22:00:00,2185-09-13 16:00:00,MAIN,Levofloxacin,...,None,750,mg,1,TAB,0.0,PO/NG,750.0,mg,Tablet
1,10959054,22224110,63864197,10959054-377,377.0,P38G1F,2185-09-11 22:00:00,2185-09-13 16:00:00,MAIN,Dextrose 50%,...,None,12.5,gm,0.5,SYR,NaN,IV,50.0,mL,Syringe
2,10959054,22224110,70038615,10959054-385,385.0,P38G1F,2185-09-11 10:00:00,2185-09-13 16:00:00,MAIN,Aspirin,...,None,81,mg,1,TAB,1.0,PO/NG,81.0,mg,Tab
3,10959054,22224110,77916045,10959054-389,389.0,P38G1F,2185-09-11 08:00:00,2185-09-13 16:00:00,MAIN,Insulin,...,None,24,UNIT,0.024,VIAL,1.0,SC,10.0,mL,Vial
4,10959054,22224110,77934658,10959054-363,363.0,P21L4X,2185-09-11 20:00:00,2185-09-11 21:00:00,BASE,0.45% Sodium Chloride,...,None,1000,mL,1000,mL,NaN,IV,1000.0,mL,Bag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,11140481,21280495,40224813,11140481-191,191.0,P573H3,2186-02-16 21:00:00,2186-02-17 12:00:00,BASE,0.9% Sodium Chloride,...,None,250,mL,250,mL,1.0,IV,250.0,mL,Bag
199996,11140481,21280495,40616531,11140481-200,200.0,P18EEH,2186-02-19 12:00:00,2186-02-18 19:00:00,MAIN,Dexamethasone,...,None,2,mg,1,TAB,4.0,PO,2.0,mg,Tab
199997,11140481,21280495,47270347,11140481-183,183.0,P47KXQ,2186-02-16 20:00:00,2186-02-18 19:00:00,MAIN,Senna,...,None,2,TAB,2,TAB,NaN,PO/NG,1.0,Table,t
199998,11140481,21280495,53949796,11140481-185,185.0,P47KXQ,2186-02-16 20:00:00,2186-02-18 19:00:00,MAIN,Docusate Sodium,...,None,100,mg,1,CAP,2.0,PO,100.0,mg,Capsule


In [2]:
df[df['dose_val_rx'].str.contains('-')]

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,quantity,unit,form
58,10959054,29665973,13891188,10959054-583,583.0,P91MW8,2186-12-17 22:00:00,2186-12-19 17:00:00,MAIN,Zolpidem Tartrate,...,None,2.5-5,mg,0.5-1,TAB,NaN,PO,5.0,mg,Tablet
63,10959054,29665973,22306696,10959054-561,561.0,P91MW8,2186-12-17 22:00:00,2186-12-20 17:00:00,MAIN,Aluminum-Magnesium Hydrox.-Simethicone,...,None,15-30,mL,0.5-1,UDCUP,NaN,PO,30.0,mL,UDCup
71,10959054,29665973,34646412,10959054-623,623.0,P70N8Q,2186-12-19 09:00:00,2186-12-20 17:00:00,MAIN,TraMADOL (Ultram),...,None,25-50,mg,0.5-1,TAB,NaN,PO,50.0,mg,Tablet
75,10959054,29665973,53335869,10959054-585,585.0,P91MW8,2186-12-17 22:00:00,2186-12-20 17:00:00,MAIN,Albuterol-Ipratropium,...,None,1-2,PUFF,0.01-0.02,CAN,NaN,IH,14.7,g,Inhaler
92,10959054,29665973,87937273,10959054-618,618.0,P90MUK,2186-12-18 21:00:00,2186-12-19 08:00:00,MAIN,OxycoDONE (Immediate Release),...,None,5-10,mg,1-2,TAB,NaN,PO,5.0,mg,Tablet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199967,11140311,27454707,3463504,11140311-74,74.0,P02L7J,2147-05-20 17:00:00,2147-05-23 18:00:00,MAIN,OxyCODONE (Immediate Release),...,None,5-10,mg,1-2,TAB,NaN,PO/NG,5.0,mg,Tablet
199968,11140311,27454707,19097944,11140311-69,69.0,P02L7J,2147-05-20 17:00:00,2147-05-23 18:00:00,MAIN,Acetaminophen,...,None,500-1000,mg,1-2,TAB,NaN,PO,500.0,mg,Tablet
199972,11140311,27454707,42295351,11140311-47,47.0,P86JLW,2147-05-20 17:00:00,2147-05-21 17:00:00,MAIN,Ketorolac,...,None,15-30,mg,1-2,VIAL,NaN,IV,15.0,mg/mL,Vial
199973,11140311,27454707,46458228,11140311-47,47.0,P86JLW,2147-05-20 17:00:00,2147-05-21 17:00:00,MAIN,Ketorolac,...,None,15-30,mg,0.5-1,VIAL,NaN,IV,30.0,mg/mL,Vial


In [3]:
df.loc[df['dose_val_rx'].str.contains('to'), 'dose_val_rx'] = df['dose_val_rx'].str.replace('to', '-')

In [4]:
df['dose_val_rx'] = df['dose_val_rx'].str.strip('-')

In [5]:
#check for ranges in dose_val_rx (e.g. 1-2)*
import numpy as np

df["dose_val_rx"] = df['dose_val_rx'].where(df['dose_val_rx'].str.contains('-'), df['dose_val_rx'].str.split('-').apply(lambda x: [float(i.replace(",",'.')) for i in x]).apply(lambda x: np.mean(x)))

ValueError: could not convert string to float: '1.62 % (4'

In [6]:
df.loc[~df['dose_val_rx'].apply(lambda x : isinstance(x, float)),"dose_val_rx"]=df[~df['dose_val_rx'].apply(lambda x : isinstance(x, float))]["dose_val_rx"].apply(lambda x: x.split('-')).apply(lambda x: [float(i) for i in x]).apply(lambda x: np.mean(x))
df.loc[~df['form_val_disp'].apply(lambda x : isinstance(x, float)),"form_val_disp"]=df[~df['form_val_disp'].apply(lambda x : isinstance(x, float))]["form_val_disp"].apply(lambda x: x.split('-')).apply(lambda x: [float(i) for i in x]).apply(lambda x: np.mean(x))

In [7]:

# check if the dose_unit_rx is a milli
df["dose_val_rx"]=df["dose_val_rx"].astype(float)
records = df.to_dict(orient='records')
for record in records:
    if isinstance(record['dose_unit_rx'], str):
        record['dose_unit_rx'] = record['dose_unit_rx'].strip()
        if record['dose_unit_rx'].startswith('m'):
            record['dose_val_rx'] = record['dose_val_rx'] / 1000
            record['dose_unit_rx'] = record['dose_unit_rx'][1:]
    if isinstance(record['form_unit_disp'], str):
        record['form_unit_disp'] = record['form_unit_disp'].strip()
        if record['form_unit_disp'].startswith('m'):
            record['form_val_disp'] = record['form_val_disp'] / 1000
            record['form_unit_disp'] = record['form_unit_disp'][1:]
            
    if isinstance(record['unit'], str):
        record['unit'] = record['unit'].strip()
        if record['unit'].startswith('m'):
            record['quantity'] = record['quantity'] / 1000
            record['unit'] = record['unit'][1:]
            if record['unit'] == 'l' : record['unit'] = 'L'
            if record['unit'] == 'G' : record['unit'] = 'g'
        if "/" in record['unit']:
            nom, denom = record['unit'].split('/')
            if denom.strip().startswith('m'):
                record['quantity'] = record['quantity'] * 1000
                denom = denom[1:]
            if denom == 'l' : denom = 'L'
            if denom == 'G' : denom = 'g'
            if nom == 'l' : nom = 'L'
            if nom == 'G' : nom = 'g'
            new_unit = nom + '/' + denom
            record['unit'] = new_unit
        
df = pd.DataFrame(records)
df[['drug','dose_val_rx', 'dose_unit_rx', 'quantity', 'unit', 'form', 'prod_strength']]


,drug,dose_val_rx,dose_unit_rx,quantity,unit,form,prod_strength
0,5% Dextrose,0.050,L,0.050,L,Bag,50 mL Bag
1,Ibuprofen,0.500,g,0.600,g,Tablet,600mg Tablet
2,Potassium Chloride,0.040,Eq,0.010,Eq,ER Tablet,10mEq ER Tablet
3,5% Dextrose,0.100,L,0.100,L,Bag,100 mL Bag
4,Doxycycline Hyclate,0.100,g,0.100,g,Vial,100 mg Vial
...,...,...,...,...,...,...,...
199995,D5 1/2NS,1.000,L,1.000,L,Bag,1000mL Bag
199996,Acetaminophen,0.650,g,0.325,g,Tablet,325mg Tablet
199997,Ondansetron,0.004,g,0.004,g,Tablet,4 mg Tablet
199998,Ibuprofen,0.800,g,0.400,g,Tablet,400mg Tablet


In [8]:
df.keys()   

Index(['subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 'poe_seq',
       'order_provider_id', 'starttime', 'stoptime', 'drug_type', 'drug',
       'formulary_drug_cd', 'gsn', 'ndc', 'prod_strength', 'form_rx',
       'dose_val_rx', 'dose_unit_rx', 'form_val_disp', 'form_unit_disp',
       'doses_per_24_hrs', 'route', 'quantity', 'unit', 'form'],
      dtype='object')

In [9]:
import seaborn as sns
way= df.groupby(['gsn'])[["unit","form"]].nunique()
way



,unit,form
gsn,,
,5,6
000018,1,1
000019,1,1
000090,1,1
000091,1,1
...,...,...
078262,1,1
078733,1,1
078791,1,1


> On va travailler sur les références de médicaments présentant 1 seule forme et 1 seule unité

In [10]:
way[(way['unit']==1) & (way['form']==1)]

,unit,form
gsn,,
000018,1,1
000019,1,1
000090,1,1
000091,1,1
000093,1,1
...,...,...
078262,1,1
078733,1,1
078791,1,1


In [11]:
work_df = df[df['gsn'].isin(way[(way['unit']==1) & (way['form']==1)].index)]

work_df

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route,quantity,unit,form
1,10791629,21819658,46855320,10791629-107,107.0,P83VKT,2171-01-16 15:00:00,2171-01-16 15:00:00,MAIN,Ibuprofen,...,None,0.500,g,0.83335,TAB,NaN,PO,0.600,g,Tablet
2,10791629,21819658,55268241,10791629-94,94.0,P83VKT,2171-01-15 19:00:00,2171-01-16 21:00:00,MAIN,Potassium Chloride,...,None,0.040,Eq,4.00000,TAB,1.0,PO,0.010,Eq,ER Tablet
4,10791629,21819658,58680366,10791629-87,87.0,P37BV6,2171-01-15 17:00:00,2171-01-16 11:00:00,MAIN,Doxycycline Hyclate,...,,0.100,g,1.00000,VIAL,2.0,IV,0.100,g,Vial
6,10791629,21819658,63026755,10791629-102,102.0,P83VKT,2171-01-16 12:00:00,2171-01-16 21:00:00,MAIN,Clindamycin,...,None,0.450,g,3.00000,CAP,4.0,PO,0.150,g,Cap
7,10791629,21819658,63405608,10791629-47,47.0,P75HZC,2171-01-14 11:00:00,2171-01-15 16:00:00,MAIN,Ibuprofen,...,None,0.600,g,1.00000,TAB,4.0,PO,0.600,g,Tablet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,10958987,28465678,96525879,10958987-41,41.0,P408EG,2124-12-22 19:00:00,2124-12-24 09:00:00,BASE,D5 1/2NS,...,None,1.000,L,1.00000,L,NaN,IV,1.000,L,Bag
199996,10959007,21338482,642943,10959007-92,92.0,P743BY,2118-10-25 02:00:00,2118-10-26 22:00:00,MAIN,Acetaminophen,...,None,0.650,g,2.00000,TAB,NaN,PO/NG,0.325,g,Tablet
199997,10959007,21338482,5851476,10959007-96,96.0,P743BY,2118-10-25 02:00:00,2118-10-26 22:00:00,MAIN,Ondansetron,...,None,0.004,g,1.00000,TAB,NaN,PO/NG,0.004,g,Tablet
199998,10959007,21338482,9918114,10959007-115,115.0,P44Z0T,2118-10-26 12:00:00,2118-10-26 22:00:00,MAIN,Ibuprofen,...,None,0.800,g,2.00000,TAB,NaN,PO,0.400,g,Tablet


In [20]:
work_df.loc[:,'dispensation_qty'] = work_df.where(work_df['dose_unit_rx'] == work_df['unit'], work_df['dose_val_rx'] / work_df['quantity'], axis=0)

> On va vérifier si le champ_form_val_disp est bien renseigné

In [21]:
work_df['dispensation_delta'] =work_df['form_val_disp'] - work_df['dispensation_qty']
work_df
work_df[(work_df['form_val_disp'] - work_df['dispensation_qty'])!=0][['prod_strength','dose_val_rx','dose_unit_rx','quantity','unit','form','form_val_disp','form_unit_disp','dispensation_qty','dispensation_delta']]

C:\Users\jvoyer\AppData\Local\Temp\5\ipykernel_14232\953967689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  work_df['dispensation_delta'] =work_df['form_val_disp'] - work_df['dispensation_qty']


,prod_strength,dose_val_rx,dose_unit_rx,quantity,unit,form,form_val_disp,form_unit_disp,dispensation_qty,dispensation_delta
1,600mg Tablet,0.500,g,0.6000,g,Tablet,0.83335,TAB,0.833333,0.000017
18,12g Inhaler,2.000,PUFF,12.0000,g,Inhaler,0.02000,INH,0.166667,-0.146667
25,8 g Inhaler,1.500,PUFF,8.0000,g,Inhaler,0.01500,INH,0.187500,-0.172500
33,16g NASAL SPRAY,2.000,SPRY,16.0000,g,NASAL SPRAY,0.02000,AERO,0.125000,-0.105000
36,50mL Syringe,12.500,gm,0.0500,L,Syringe,0.50000,SYR,250.000000,-249.500000
...,...,...,...,...,...,...,...,...,...,...
199911,2.5mL Vial,1.000,NEB,0.0025,L,Vial,1.00000,VIAL,400.000000,-399.000000
199939,2.5mL Vial,1.000,NEB,0.0025,L,Vial,1.00000,VIAL,400.000000,-399.000000
199970,1000mL Bag,1.000,L,1.0000,L,Bag,0.00100,L,1.000000,-0.999000
199979,50mg/mL Vial,0.025,g,50.0000,g/L,Vial,0.50000,VIAL,0.000500,0.499500
